In [ ]:
import utils
import importlib

from automaton import State, Automaton, ThompsonConstruction

importlib.reload(utils)

In [ ]:
infix = '(a+b)*(a+c)+a+b*c'
postfix = utils.infix2postfix(infix)
root = utils.postfix2tree(postfix)
print(root.get_postfix())

root.show_dot()

In [ ]:
def token_type(character):
    if character in ['.', '|']: return 'binary'
    elif character == '*': return 'unary'
    return False

infix = '(a|b)*abb'
infix = utils.add_explicit_concatenation(infix)
postfix = utils.infix2postfix(infix, 
                              priority={'*': 2, '.': 1, '|': 0})
root = utils.postfix2tree(postfix, token_type)
print(infix)
print(postfix)
print(root.get_postfix())

root.show_dot()

In [ ]:
expr1 = ThompsonConstruction.expr_union(ThompsonConstruction.create_expr('a'), 
                                        ThompsonConstruction.create_expr('b'))
expr2 = ThompsonConstruction.expr_star(expr1)
expr3 = ThompsonConstruction.expr_concat(expr2, ThompsonConstruction.create_expr('a'))
expr4 = ThompsonConstruction.expr_concat(expr3, ThompsonConstruction.create_expr('b'))
expr5 = ThompsonConstruction.expr_concat(expr4, ThompsonConstruction.create_expr('b'))

M = Automaton('M', _STATE_REGISTER, ['a', 'b'], expr5['initial_state'], expr5['out_state'])

In [ ]:
M.show_dot()